In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import regexp_extract

In [7]:
cluster = 'localhost:7077'
spark = SparkSession \
        .builder \
        .master('spark://'+cluster) \
        .appName('AirFlow Log Test') \
        .getOrCreate()
#        .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
# spark.stop()

In [8]:
import pprint
pprint.pprint(spark.sparkContext.getConf().getAll())
# spark.stop()

[('spark.driver.port', '51033'),
 ('spark.app.startTime', '1644310231611'),
 ('spark.app.name', 'AirFlow Log Test'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/miniporject%232/code/spark-warehouse'),
 ('spark.master', 'spark://localhost:7077'),
 ('spark.app.id', 'app-20220208085033-0000'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '172.30.1.26'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]


In [9]:
# print("Spark Object id created ...")
# print("Number of partitions for shuffle changed to : " + str(spark.conf.get('spark.sql.shuffle.partitions')))
# # print("The HDFS Path is " + spark.conf.get("spark.yarn.appMasterEnv.HDFS_PATH"))
# print("The HDFS Path is " + spark.conf.get("spark.hadoop.conf"))
# spark.stop()

In [10]:
# base_df_rdd = spark.read.text("hdfs://localhost:9000/user/root/input").rdd
# base_df_rdd = spark.read.text("hdfs://localhost:9000/user/root/input/MySQL_to_CSV.py.log").rdd
# base_df = spark.read.text("hdfs://localhost:9000/user/root/input/MySQL_to_CSV.py.log")
path = "hdfs://localhost:9870/user/root/input/MySQL_to_CSV.py.log"
# base_df = spark.sparkContext.textFile(path)
base_df = spark.read.text(path)
type(base_df)

22/02/08 19:41:56 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://localhost:9870/user/root/input/MySQL_to_CSV.py.log.
org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1906)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1202)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1098)


Py4JJavaError: An error occurred while calling o118.text.
: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1906)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1202)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1098)


In [16]:
for i in base_df.collect():
    print(i)

hdfs://namenode:9870/user/root/input/MySQL_to_CSV.py.log MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0


22/02/07 00:29:48 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() for one-way message.
java.io.InvalidClassException: org.apache.spark.deploy.DeployMessages$ExecutorUpdated; local class incompatible: stream classdesc serialVersionUID = 1654279024112373855, local class serialVersionUID = -1971851081955655249
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:699)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2003)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1850)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2160)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:503)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:461)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:76)
	at org.apache.spark.serializer.JavaSerializerInstance.deserializ

In [14]:
log_pattern =r"^\[(\d+-\d+-\d+\s\d+:\d+:\d+,\d+)\] \{\S+\} (DEBUG|INFO|WARN|FATAL|ERROR|TRACE)([^][]*)|(.*)"# ([^][]*)|([^][]*)([^][]*)(\'\S+)"
logs_df = base_df.select(regexp_extract('value', log_pattern, 1).alias('datetime'),
                     regexp_extract('value', log_pattern, 2).alias('log-level'),
                       regexp_extract('value', log_pattern, 3).alias('message')                     
                     )
logs_df.show(truncate=True)
print((logs_df.count(), len(logs_df.columns)))


AttributeError: 'RDD' object has no attribute 'select'

In [24]:
log_df.show()

22/02/04 01:39:01 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/04 01:39:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/04 01:39:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/04 01:39:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/04 01:40:01 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/04 01:40:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

KeyboardInterrupt: 

In [26]:
spark.stop()